# CLM6-BGC OAAT parameter ensemble
- generates paramfiles and namelist_mods for the OAAT experiments
- leverages ppe-tools to write out the files
- written with community-derived ensemble ranges:
    -https://docs.google.com/spreadsheets/d/1R0AybNR0YAmMDjRqp9oyUffDhKeAWv1QF4yWTHqiXXM/edit?usp=drive_link
- Katie Dagon (kdagon@ucar.edu) and Daniel Kennedy (djk2120@ucar.edu) and Linnia Hawkins (lh3194@columbia.edu)

In [1]:
import numpy as np
from ppe_tools import Ensemble,Member,ParamInfo
from ppe_tools.utils import get_default, parse_val
import xarray as xr
import os
import pandas as pd
import glob

In [2]:
# Load file with parameter ranges
csv='clm6oaat_paramranges_10252024.csv'
params = pd.read_csv(csv)
params = params[params['include'] == 1]

In [5]:
# default parameter file to start with
basefile = '/glade/work/linnia/ctsm5.3.0/cime/scripts/transient/runtime_files/ctsm60_params.c241007.nc'
# where the parameter files and namelist mods should be written to:
pdir = '/glade/work/linnia/CLM6-PPE/CLM6_miniOAAT/paramfiles/'
ndir = '/glade/work/linnia/CLM6-PPE/CLM6_miniOAAT/namelist_mods/'
#instantiate the Ensemble object
x    = Ensemble(basefile,pdir,ndir)

In [3]:
#create dictionary for the independent parameters 
oaats = {}
names = params['name']
flags = params['flag']
for name,flag in zip(names,flags):
        if not pd.notnull(flag):
            ix = params['name']==name
            minval = params['min'][ix].values[0]
            maxval = params['max'][ix].values[0]
            pftmin = params['pft_mins'][ix].values[0]
            pftmax = params['pft_maxs'][ix].values[0]
            thisloc = params['loc'][ix].values[0]

            needs_pft = (minval=='pft')
            if needs_pft:
                thismin = np.zeros((79))
                thismin[0:17] = np.fromstring(pftmin, dtype='float', sep=',')
            elif 'percent' in minval:
                thismin = minval
            else:
                thismin = np.array(float(minval))

            needs_pft = (maxval=='pft')
            if needs_pft:
                thismax = np.zeros((79))
                thismax[0:17] = np.fromstring(pftmax, dtype='float', sep=',')
            elif 'percent' in maxval:
                thismax = maxval
            else:
                thismax = np.array(float(maxval))

            oaats[name]={'min':thismin,'max':thismax,'loc':thisloc}


In [6]:
#add to the Ensemble object
prefix = 'oaat'
nextnum=0000
x.add_oaats(oaats,prefix,nextnum,skipBFB=True)

interception_fraction-max looks BFB.... skipping
accum_factor-min looks BFB.... skipping
snw_rds_refrz-max looks BFB.... skipping
decomp_depth_efolding-max looks BFB.... skipping
bgc_rf_s2s1-min looks BFB.... skipping


In [ ]:
#add params that move in unison to the Ensemble object
flags = params['flag']
ix = pd.notnull(params['flag'])
uflags = pd.unique(flags[ix])
sgns = {'min':'-','max':''}
for uflag in uflags:
    names = params['name'][flags==uflag]
    for minmax in ['min','max']:
        mf = {}
        
        for name in names:
            ix = params['name']==name

            thisval = params[minmax][ix].values[0]
            pftval  = params['pft_'+minmax+'s'][ix].values[0]
            thisloc = params['loc'][ix].values[0]

            needs_pft = (thisval=='pft')
            if needs_pft:
                val = np.fromstring(pftval, dtype='float', sep=',')
            elif 'percent' in thisval:
                val = sgns[minmax]+thisval
            else:
                val = np.array(float(thisval))

            mf[name] = {'value':val,'loc':thisloc,'minmax':minmax,'flag':uflag}

        x.add_mf(mf,'oaat')


In [7]:
# write the parameter files and namelist mods 
csvfile= '/glade/work/linnia/CLM6-PPE/CLM6_miniOAAT/clm6_minioaat.csv'
x.write(oaatfile=csvfile)